**Submission by**

- Naman Dudhoria         roll no.36
- Deepa Rawat            roll no.24
- Gouri Mangal           roll no.29
- Avimalya Dey           roll no.18

    IIFT Kolkata
# Stock Market Prediction Project
## Part 1: Support Vector Classifier for Stock Buying Decisions
## Part 2: ARIMA, SVR, XGBoost, and Neural Networks for Stock Price Prediction

**Project Overview:**
- Analyze 10 stocks from different industries
- Implement SVC for buying decisions
- Compare SARIMAX, SVR, XGBoost, and Neural Networks for price prediction
- Create comprehensive visualizations using Plotly
- Build equal-weight portfolio and compare with market indices

**Stocks Analyzed:**
1. HDFC Bank (Banking)
2. TCS (IT Services)
3. Maruti Suzuki (Automobile)
4. Asian Paints (Manufacturing)
5. Dabur (FMCG)
6. Dr. Reddy's (Healthcare)
7. Apollo Hospitals (Healthcare)
8. Airtel (Telecom)
9. Mazagon Dock (Shipyard)
10. Motilal Oswal (NBFC)


In [1]:
# Importing all necessary libraries
import sys
print(sys.executable)
print(sys.version)
!{sys.executable} -m pip install tensorflow --timeout=120
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
from datetime import datetime, timedelta
import os
import glob

print("All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


c:\Users\home\miniconda3\python.exe
3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)]


All libraries imported successfully!
TensorFlow version: 2.20.0
Pandas version: 2.1.3
NumPy version: 1.26.2


## Data Loading and Preprocessing

In this section, we will:
1. Load all 10 stock datasets
2. Clean and standardize the data format
3. Handle missing values and outliers
4. Create consistent date indexing
5. Prepare data for both classification (buying decisions) and regression (price prediction) tasks


In [2]:

stocks_info = {
    'HDFC_BANK.csv': {'name': 'HDFC Bank', 'sector': 'Banking'},
    'TCS_IT.csv': {'name': 'TCS', 'sector': 'IT Services'},
    'Maruti_automobile.csv': {'name': 'Maruti Suzuki', 'sector': 'Automobile'},
    'Asianpaints_manufacturing.csv': {'name': 'Asian Paints', 'sector': 'Manufacturing'},
    'Dabur_fmcg.csv': {'name': 'Dabur', 'sector': 'FMCG'},
    'Dr.reddy_healthcare.csv': {'name': 'Dr. Reddy\'s', 'sector': 'Healthcare'},
    'APOLLO_Hospitals.csv': {'name': 'Apollo Hospitals', 'sector': 'Healthcare'},
    'Airtel_telecom.csv': {'name': 'Airtel', 'sector': 'Telecom'},
    'Mazagon_shipyard.csv': {'name': 'Mazagon Dock', 'sector': 'Shipyard'},
    'Motilal_oswal.csv': {'name': 'Motilal Oswal', 'sector': 'NBFC'}
}


def load_stock_data(filename, stock_name, sector):
    try:
        df = pd.read_csv(filename)
        

        print(f"\n=== {stock_name} ({sector}) ===")
        print(f"Original shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        df.columns = df.columns.str.strip()
        
        column_mapping = {
            'Date': 'Date',
            'Open Price': 'OPEN',
            'High Price': 'HIGH', 
            'Low Price': 'LOW',
            'Close Price': 'CLOSE',
            'Last Price': 'CLOSE',  
            'Total Traded Quantity': 'VOLUME'
        }
        
        for old_col, new_col in column_mapping.items():
            if old_col in df.columns and new_col not in df.columns:
                df[new_col] = df[old_col]
        
        
        required_cols = ['Date', 'OPEN', 'HIGH', 'LOW', 'CLOSE']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if missing_cols:
            print(f"Warning: Missing columns {missing_cols} in {filename}")
            return None
        
        
        df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')
        
        
        df = df.dropna(subset=['Date'])
        
        
        price_cols = ['OPEN', 'HIGH', 'LOW', 'CLOSE']
        for col in price_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).str.replace(',', '').astype(float)
        
       
        df = df.sort_values('Date').reset_index(drop=True)
        
        
        df = df.drop_duplicates(subset=['Date']).reset_index(drop=True)
        
        
        df['STOCK_NAME'] = stock_name
        df['SECTOR'] = sector
        

        df['RETURNS'] = df['CLOSE'].pct_change()
        df['VOLATILITY'] = df['RETURNS'].rolling(window=20).std()
        df['SMA_20'] = df['CLOSE'].rolling(window=20).mean()
        df['SMA_50'] = df['CLOSE'].rolling(window=50).mean()
        
        print(f"Processed shape: {df.shape}")
        print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
        print(f"Missing values: {df[price_cols].isnull().sum().sum()}")
        
        return df
        
    except Exception as e:
        print(f"Error loading {filename}: {str(e)}")
        return None


stocks_data = {}
for filename, info in stocks_info.items():
    if os.path.exists(filename):
        df = load_stock_data(filename, info['name'], info['sector'])
        if df is not None:
            stocks_data[info['name']] = df
    else:
        print(f"File not found: {filename}")

print(f"\nSuccessfully loaded {len(stocks_data)} stocks")
print("Available stocks:", list(stocks_data.keys()))



=== HDFC Bank (Banking) ===
Original shape: (773, 15)
Columns: ['Symbol  ', 'Series  ', 'Date  ', 'Prev Close  ', 'Open Price  ', 'High Price  ', 'Low Price  ', 'Last Price  ', 'Close Price  ', 'Average Price ', 'Total Traded Quantity  ', 'Turnover ₹  ', 'No. of Trades  ', 'Deliverable Qty  ', '% Dly Qt to Traded Qty  ']
Processed shape: (746, 26)
Date range: 2022-09-26 00:00:00 to 2025-09-25 00:00:00
Missing values: 0

=== TCS (IT Services) ===
Original shape: (754, 15)
Columns: ['Symbol  ', 'Series  ', 'Date  ', 'Prev Close  ', 'Open Price  ', 'High Price  ', 'Low Price  ', 'Last Price  ', 'Close Price  ', 'Average Price ', 'Total Traded Quantity  ', 'Turnover ₹  ', 'No. of Trades  ', 'Deliverable Qty  ', '% Dly Qt to Traded Qty  ']
Processed shape: (746, 26)
Date range: 2022-09-26 00:00:00 to 2025-09-25 00:00:00
Missing values: 0

=== Maruti Suzuki (Automobile) ===
Original shape: (248, 15)
Columns: ['Symbol  ', 'Series  ', 'Date  ', 'Prev Close  ', 'Open Price  ', 'High Price  ', 

In [3]:
# Data Preparation for Machine Learning Models

def prepare_features_and_target(df, stock_name):
    df_processed = df.copy()
    df_processed.index = pd.to_datetime(df_processed['Date'])
    df_processed = df_processed.drop(['Date'], axis='columns')
    df_processed['Open-Close'] = df_processed['OPEN'] - df_processed['CLOSE']
    df_processed['High-Low'] = df_processed['HIGH'] - df_processed['LOW']
    X = df_processed[['Open-Close', 'High-Low']].copy()
    y = np.where(df_processed['CLOSE'].shift(-1) > df_processed['CLOSE'], 1, 0)
    X = X[:-1]
    y = y[:-1]
    
    print(f"\n{stock_name} - Features and Target prepared:")
    print(f"Features shape: {X.shape}")
    print(f"Target shape: {y.shape}")
    print(f"Buy signals (1): {np.sum(y == 1)}")
    print(f"No position signals (0): {np.sum(y == 0)}")
    
    return X, y, df_processed
    
stocks_features = {}
stocks_targets = {}
stocks_processed = {}

for stock_name, df in stocks_data.items():
    X, y, df_proc = prepare_features_and_target(df, stock_name)
    stocks_features[stock_name] = X
    stocks_targets[stock_name] = y
    stocks_processed[stock_name] = df_proc

print(f"\nData preparation completed for {len(stocks_features)} stocks")



HDFC Bank - Features and Target prepared:
Features shape: (745, 2)
Target shape: (745,)
Buy signals (1): 390
No position signals (0): 355

TCS - Features and Target prepared:
Features shape: (745, 2)
Target shape: (745,)
Buy signals (1): 355
No position signals (0): 390

Maruti Suzuki - Features and Target prepared:
Features shape: (246, 2)
Target shape: (246,)
Buy signals (1): 121
No position signals (0): 125

Asian Paints - Features and Target prepared:
Features shape: (745, 2)
Target shape: (745,)
Buy signals (1): 375
No position signals (0): 370

Dabur - Features and Target prepared:
Features shape: (745, 2)
Target shape: (745,)
Buy signals (1): 372
No position signals (0): 373

Dr. Reddy's - Features and Target prepared:
Features shape: (745, 2)
Target shape: (745,)
Buy signals (1): 395
No position signals (0): 350

Apollo Hospitals - Features and Target prepared:
Features shape: (746, 2)
Target shape: (746,)
Buy signals (1): 367
No position signals (0): 379

Airtel - Features an

## Part 1: Support Vector Classifier for Stock Buying Decisions

In this section, we will:
1. Implement SVC for each stock to predict buying decisions
2. Use TimeSeriesSplit for proper time series cross-validation
3. Compare cumulative returns from SVC predictions vs actual returns
4. Create visualizations showing the performance


In [4]:
# Part 1: Support Vector Classifier Implementation

def train_svc_model(X, y, stock_name, test_size=0.2):
    print(f"\n=== Training SVC for {stock_name} ===")
    
    split_percentage = 1 - test_size
    split = int(split_percentage * len(X))
    
    X_train = X[:split]
    y_train = y[:split]
    
    X_test = X[split:]
    y_test = y[split:]
    
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    svc = SVC(kernel='rbf', random_state=42)
    
    svc.fit(X_train_scaled, y_train)
    
    y_pred_test = svc.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred_test)
    
    print(f"SVC Accuracy: {accuracy:.4f}")
    print(f"Test predictions sample: {y_pred_test[:5]}")
    
    return {
        'model': svc,
        'scaler': scaler,
        'y_test': y_test,
        'y_pred_test': y_pred_test,
        'accuracy': accuracy,
        'X_test': X_test,
        'X_train': X_train,
        'y_train': y_train
    }
svc_results = {}

for stock_name in stocks_features.keys():
    X = stocks_features[stock_name]
    y = stocks_targets[stock_name]
    valid_indices = ~(X.isnull().any(axis=1) | pd.isnull(y))
    X_clean = X[valid_indices]
    y_clean = y[valid_indices]
    
    if len(X_clean) > 100: 
        result = train_svc_model(X_clean, y_clean, stock_name)
        svc_results[stock_name] = result
    else:
        print(f"Skipping {stock_name} - insufficient data")

print(f"\nSVC training completed for {len(svc_results)} stocks")



=== Training SVC for HDFC Bank ===
Training set size: 596
Test set size: 149
SVC Accuracy: 0.5235
Test predictions sample: [1 1 1 0 1]

=== Training SVC for TCS ===
Training set size: 596
Test set size: 149
SVC Accuracy: 0.5705
Test predictions sample: [1 1 1 0 0]

=== Training SVC for Maruti Suzuki ===
Training set size: 196
Test set size: 50
SVC Accuracy: 0.4000
Test predictions sample: [0 0 0 0 0]

=== Training SVC for Asian Paints ===
Training set size: 596
Test set size: 149
SVC Accuracy: 0.5235
Test predictions sample: [1 0 0 0 0]

=== Training SVC for Dabur ===
Training set size: 596
Test set size: 149
SVC Accuracy: 0.4966
Test predictions sample: [1 0 1 0 1]

=== Training SVC for Dr. Reddy's ===
Training set size: 596
Test set size: 149
SVC Accuracy: 0.5436
Test predictions sample: [1 1 1 1 1]

=== Training SVC for Apollo Hospitals ===
Training set size: 596
Test set size: 150
SVC Accuracy: 0.4733
Test predictions sample: [0 0 0 1 1]

=== Training SVC for Airtel ===
Training s

In [5]:
# Calculate Cumulative Returns for SVC Predictions vs Actual Returns

def calculate_cumulative_returns(stock_name, svc_result, stock_data):
    
    test_size = len(svc_result['y_test'])
    total_size = len(stock_data)
    train_size = total_size - test_size
    
    test_prices = stock_data['CLOSE'].iloc[train_size:train_size + test_size].values
    test_dates = stock_data['Date'].iloc[train_size:train_size + test_size].values
    actual_returns = np.diff(test_prices) / test_prices[:-1]
    svc_predictions = svc_result['y_pred_test']
    svc_returns = np.where(svc_predictions[:-1] == 1, actual_returns, 0)
    actual_cumulative = np.cumprod(1 + actual_returns) - 1
    svc_cumulative = np.cumprod(1 + svc_returns) - 1
    buy_hold_returns = actual_returns
    buy_hold_cumulative = np.cumprod(1 + buy_hold_returns) - 1
    
    return {
        'stock_name': stock_name,
        'dates': test_dates[1:], 
        'actual_cumulative': actual_cumulative,
        'svc_cumulative': svc_cumulative,
        'buy_hold_cumulative': buy_hold_cumulative,
        'actual_returns': actual_returns,
        'svc_returns': svc_returns,
        'svc_predictions': svc_predictions[:-1]  
    }

cumulative_returns = {}

for stock_name in svc_results.keys():
    if stock_name in stocks_data:
        result = calculate_cumulative_returns(stock_name, svc_results[stock_name], stocks_data[stock_name])
        cumulative_returns[stock_name] = result
        
        # Print summary
        final_actual = result['actual_cumulative'][-1]
        final_svc = result['svc_cumulative'][-1]
        final_buy_hold = result['buy_hold_cumulative'][-1]
        
        print(f"\n{stock_name} - Cumulative Returns:")
        print(f"Actual Returns: {final_actual:.4f} ({final_actual*100:.2f}%)")
        print(f"SVC Strategy: {final_svc:.4f} ({final_svc*100:.2f}%)")
        print(f"Buy & Hold: {final_buy_hold:.4f} ({final_buy_hold*100:.2f}%)")
        print(f"SVC vs Buy & Hold: {final_svc - final_buy_hold:.4f} ({(final_svc - final_buy_hold)*100:.2f}%)")

print(f"\nCumulative returns calculated for {len(cumulative_returns)} stocks")



HDFC Bank - Cumulative Returns:
Actual Returns: -0.4487 (-44.87%)
SVC Strategy: 0.1212 (12.12%)
Buy & Hold: -0.4487 (-44.87%)
SVC vs Buy & Hold: 0.5699 (56.99%)

TCS - Cumulative Returns:
Actual Returns: -0.2364 (-23.64%)
SVC Strategy: -0.0619 (-6.19%)
Buy & Hold: -0.2364 (-23.64%)
SVC vs Buy & Hold: 0.1746 (17.46%)

Maruti Suzuki - Cumulative Returns:
Actual Returns: 0.2949 (29.49%)
SVC Strategy: -0.0245 (-2.45%)
Buy & Hold: 0.2949 (29.49%)
SVC vs Buy & Hold: -0.3194 (-31.94%)

Asian Paints - Cumulative Returns:
Actual Returns: 0.0744 (7.44%)
SVC Strategy: -0.0561 (-5.61%)
Buy & Hold: 0.0744 (7.44%)
SVC vs Buy & Hold: -0.1305 (-13.05%)

Dabur - Cumulative Returns:
Actual Returns: -0.0172 (-1.72%)
SVC Strategy: -0.1780 (-17.80%)
Buy & Hold: -0.0172 (-1.72%)
SVC vs Buy & Hold: -0.1608 (-16.08%)

Dr. Reddy's - Cumulative Returns:
Actual Returns: 0.0650 (6.50%)
SVC Strategy: 0.0650 (6.50%)
Buy & Hold: 0.0650 (6.50%)
SVC vs Buy & Hold: 0.0000 (0.00%)

Apollo Hospitals - Cumulative Returns

In [6]:
def create_equal_weight_portfolio(cumulative_returns_data):
    stock_names = list(cumulative_returns_data.keys())
    n_stocks = len(stock_names)
    min_length = min(len(data['svc_cumulative']) for data in cumulative_returns_data.values())
    portfolio_svc_returns = np.zeros(min_length)
    portfolio_buy_hold_returns = np.zeros(min_length)
    portfolio_actual_returns = np.zeros(min_length)
    for stock_name, data in cumulative_returns_data.items():
        svc_returns = data['svc_returns'][:min_length]
        buy_hold_returns = data['actual_returns'][:min_length]
        actual_returns = data['actual_returns'][:min_length]
        portfolio_svc_returns += svc_returns / n_stocks
        portfolio_buy_hold_returns += buy_hold_returns / n_stocks
        portfolio_actual_returns += actual_returns / n_stocks

    portfolio_svc_cumulative = np.cumprod(1 + portfolio_svc_returns) - 1
    portfolio_buy_hold_cumulative = np.cumprod(1 + portfolio_buy_hold_returns) - 1
    portfolio_actual_cumulative = np.cumprod(1 + portfolio_actual_returns) - 1
    first_stock_data = list(cumulative_returns_data.values())[0]
    portfolio_dates = first_stock_data['dates'][:min_length]
    
    return {
        'dates': portfolio_dates,
        'portfolio_svc_cumulative': portfolio_svc_cumulative,
        'portfolio_buy_hold_cumulative': portfolio_buy_hold_cumulative,
        'portfolio_actual_cumulative': portfolio_actual_cumulative,
        'portfolio_svc_returns': portfolio_svc_returns,
        'portfolio_buy_hold_returns': portfolio_buy_hold_returns,
        'portfolio_actual_returns': portfolio_actual_returns,
        'stock_names': stock_names,
        'n_stocks': n_stocks
    }

if cumulative_returns:
    portfolio_data = create_equal_weight_portfolio(cumulative_returns)
    final_svc = portfolio_data['portfolio_svc_cumulative'][-1]
    final_buy_hold = portfolio_data['portfolio_buy_hold_cumulative'][-1]
    final_actual = portfolio_data['portfolio_actual_cumulative'][-1]
    
    print(f"\n=== Equal-Weight Portfolio Performance ===")
    print(f"Number of stocks: {portfolio_data['n_stocks']}")
    print(f"Stocks included: {', '.join(portfolio_data['stock_names'])}")
    print(f"\nPortfolio Cumulative Returns:")
    print(f"SVC Strategy: {final_svc:.4f} ({final_svc*100:.2f}%)")
    print(f"Buy & Hold: {final_buy_hold:.4f} ({final_buy_hold*100:.2f}%)")
    print(f"Actual Returns: {final_actual:.4f} ({final_actual*100:.2f}%)")
    print(f"SVC vs Buy & Hold: {final_svc - final_buy_hold:.4f} ({(final_svc - final_buy_hold)*100:.2f}%)")
    
    portfolio_volatility = np.std(portfolio_data['portfolio_svc_returns']) * np.sqrt(252)
    portfolio_sharpe = (final_svc * 252) / (portfolio_volatility * np.sqrt(252)) if portfolio_volatility > 0 else 0
    
    print(f"\nPortfolio Statistics:")
    print(f"Annualized Volatility: {portfolio_volatility:.4f} ({portfolio_volatility*100:.2f}%)")
    print(f"Sharpe Ratio: {portfolio_sharpe:.4f}")
else:
    print("No cumulative returns data available for portfolio creation")



=== Equal-Weight Portfolio Performance ===
Number of stocks: 9
Stocks included: HDFC Bank, TCS, Maruti Suzuki, Asian Paints, Dabur, Dr. Reddy's, Apollo Hospitals, Airtel, Motilal Oswal

Portfolio Cumulative Returns:
SVC Strategy: 0.0088 (0.88%)
Buy & Hold: 0.0633 (6.33%)
Actual Returns: 0.0633 (6.33%)
SVC vs Buy & Hold: -0.0545 (-5.45%)

Portfolio Statistics:
Annualized Volatility: 0.0831 (8.31%)
Sharpe Ratio: 1.6745


In [7]:
# Create Comprehensive Visualizations using Plotly

def create_svc_performance_visualization(cumulative_returns_data, portfolio_data):
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            'Individual Stock SVC vs Buy & Hold Returns',
            'Portfolio Performance Comparison',
            'SVC Prediction Accuracy by Stock',
            'Cumulative Returns Over Time',
            'Return Distribution Comparison',
            'Portfolio Risk-Return Analysis'
        ],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    stock_names = list(cumulative_returns_data.keys())
    svc_final_returns = []
    buy_hold_final_returns = []
    
    for stock_name in stock_names:
        data = cumulative_returns_data[stock_name]
        svc_final_returns.append(data['svc_cumulative'][-1] * 100)
        buy_hold_final_returns.append(data['buy_hold_cumulative'][-1] * 100)
    
    fig.add_trace(
        go.Bar(name='SVC Strategy', x=stock_names, y=svc_final_returns, 
               marker_color='lightblue', showlegend=False),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(name='Buy & Hold', x=stock_names, y=buy_hold_final_returns,
               marker_color='lightcoral', showlegend=False),
        row=1, col=1
    )
    if portfolio_data:
        dates = portfolio_data['dates']
        portfolio_svc = portfolio_data['portfolio_svc_cumulative'] * 100
        portfolio_buy_hold = portfolio_data['portfolio_buy_hold_cumulative'] * 100
        
        fig.add_trace(
            go.Scatter(x=dates, y=portfolio_svc, mode='lines', name='Portfolio SVC',
                      line=dict(color='blue', width=2)),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Scatter(x=dates, y=portfolio_buy_hold, mode='lines', name='Portfolio Buy & Hold',
                      line=dict(color='red', width=2)),
            row=1, col=2
        )

    accuracies = []
    for stock_name in stock_names:
        if stock_name in svc_results:
            accuracies.append(svc_results[stock_name]['accuracy'] * 100)
        else:
            accuracies.append(0)
    
    fig.add_trace(
        go.Bar(x=stock_names, y=accuracies, marker_color='green', showlegend=False),
        row=2, col=1
    )
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for i, (stock_name, data) in enumerate(list(cumulative_returns_data.items())[:5]):
        if i < len(colors):
            fig.add_trace(
                go.Scatter(x=data['dates'], y=data['svc_cumulative'] * 100,
                          mode='lines', name=f'{stock_name} SVC',
                          line=dict(color=colors[i], width=1)),
                row=2, col=2
            )
    if portfolio_data:
        svc_returns = portfolio_data['portfolio_svc_returns']
        buy_hold_returns = portfolio_data['portfolio_buy_hold_returns']
        
        fig.add_trace(
            go.Histogram(x=svc_returns, name='SVC Returns', opacity=0.7,
                        marker_color='blue', showlegend=False),
            row=3, col=1
        )
        
        fig.add_trace(
            go.Histogram(x=buy_hold_returns, name='Buy & Hold Returns', opacity=0.7,
                        marker_color='red', showlegend=False),
            row=3, col=1
        )
    

    if portfolio_data:
        risks = []
        returns = []
        
        for stock_name, data in cumulative_returns_data.items():
            stock_returns = data['svc_returns']
            risk = np.std(stock_returns) * np.sqrt(252) * 100  # Annualized volatility
            return_val = data['svc_cumulative'][-1] * 100  # Total return
            
            risks.append(risk)
            returns.append(return_val)
        
        fig.add_trace(
            go.Scatter(x=risks, y=returns, mode='markers+text',
                      text=stock_names, textposition="top center",
                      marker=dict(size=10, color='blue'),
                      showlegend=False),
            row=3, col=2
        )
    fig.update_layout(
        title_text="SVC Stock Market Prediction Performance Analysis",
        title_x=0.5,
        height=1200,
        showlegend=True
    )
    fig.update_xaxes(title_text="Stocks", row=1, col=1)
    fig.update_yaxes(title_text="Returns (%)", row=1, col=1)
    
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_yaxes(title_text="Cumulative Returns (%)", row=1, col=2)
    
    fig.update_xaxes(title_text="Stocks", row=2, col=1)
    fig.update_yaxes(title_text="Accuracy (%)", row=2, col=1)
    
    fig.update_xaxes(title_text="Date", row=2, col=2)
    fig.update_yaxes(title_text="Cumulative Returns (%)", row=2, col=2)
    
    fig.update_xaxes(title_text="Daily Returns", row=3, col=1)
    fig.update_yaxes(title_text="Frequency", row=3, col=1)
    
    fig.update_xaxes(title_text="Risk (Volatility %)", row=3, col=2)
    fig.update_yaxes(title_text="Return (%)", row=3, col=2)
    
    fig.show()
if cumulative_returns and 'portfolio_data' in locals():
    create_svc_performance_visualization(cumulative_returns, portfolio_data)
else:
    print("Insufficient data for visualization")


## Part 2: ARIMA, SVR, XGBoost, and Neural Networks for Stock Price Prediction

In this section, we will:
1. Implement SARIMAX for time series forecasting
2. Use Support Vector Regression (SVR) for price prediction
3. Apply XGBoost for enhanced prediction accuracy
4. Build Neural Networks (LSTM/GRU) for deep learning approach
5. Compare all models using MSE, MAE, and R² metrics
6. Create comprehensive visualizations


In [8]:
# Part 2: SARIMAX Implementation for Stock Price Prediction

def prepare_sarimax_data(df, stock_name, target_col='CLOSE'):
    
    df_processed = df.copy()
    df_processed.index = pd.to_datetime(df_processed['Date'])
    
    df_processed['Open-Close'] = df_processed['OPEN'] - df_processed['CLOSE']
    df_processed['High-Low'] = df_processed['HIGH'] - df_processed['LOW']
    df_processed['Volume'] = df_processed.get('VOLUME', 1)  # Use volume if available, else 1

    exog_vars = ['Open-Close', 'High-Low', 'Volume']
    exog_data = df_processed[exog_vars].fillna(method='ffill').fillna(0)
    
    target_data = df_processed[target_col].dropna()
    
    common_index = target_data.index.intersection(exog_data.index)
    target_data = target_data.loc[common_index]
    exog_data = exog_data.loc[common_index]
    split_point = int(0.8 * len(target_data))
    
    train_target = target_data[:split_point]
    test_target = target_data[split_point:]
    train_exog = exog_data[:split_point]
    test_exog = exog_data[split_point:]
    
    print(f"{stock_name} - SARIMAX Data Preparation:")
    print(f"Training period: {train_target.index[0]} to {train_target.index[-1]}")
    print(f"Testing period: {test_target.index[0]} to {test_target.index[-1]}")
    print(f"Training samples: {len(train_target)}")
    print(f"Testing samples: {len(test_target)}")
    
    return train_target, test_target, train_exog, test_exog

def fit_sarimax_model(train_target, train_exog, stock_name):
    try:
        model = SARIMAX(
            train_target,
            exog=train_exog,
            order=(1, 1, 1),  # (p, d, q)
            seasonal_order=(1, 1, 1, 12),  # (P, D, Q, s) - monthly seasonality
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        
        fitted_model = model.fit(disp=False, maxiter=50)
        
        print(f"{stock_name} - SARIMAX Model fitted successfully")
        print(f"AIC: {fitted_model.aic:.2f}")
        print(f"BIC: {fitted_model.bic:.2f}")
        
        return fitted_model
        
    except Exception as e:
        print(f"{stock_name} - SARIMAX Model failed: {str(e)}")
        try:
            model = SARIMAX(
                train_target,
                order=(1, 1, 1),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            fitted_model = model.fit(disp=False, maxiter=50)
            print(f"{stock_name} - Simple SARIMAX Model fitted successfully")
            return fitted_model
        except Exception as e2:
            print(f"{stock_name} - Simple SARIMAX also failed: {str(e2)}")
            return None

def predict_sarimax(model, test_exog, steps):
    try:
        if model is None:
            return np.full(steps, np.nan)
        
        predictions = model.forecast(steps=steps, exog=test_exog)
        return predictions.values if hasattr(predictions, 'values') else predictions
        
    except Exception as e:
        print(f"SARIMAX prediction error: {str(e)}")
        return np.full(steps, np.nan)

sarimax_results = {}

for stock_name, df in stocks_data.items():
    print(f"\n{'='*50}")
    print(f"Processing {stock_name} for SARIMAX")
    print(f"{'='*50}")
    
    train_target, test_target, train_exog, test_exog = prepare_sarimax_data(df, stock_name)
    
    model = fit_sarimax_model(train_target, train_exog, stock_name)
    
    if model is not None:
        predictions = predict_sarimax(model, test_exog, len(test_target))
        
        mse = mean_squared_error(test_target, predictions)
        mae = mean_absolute_error(test_target, predictions)
        r2 = r2_score(test_target, predictions)
        
        sarimax_results[stock_name] = {
            'model': model,
            'predictions': predictions,
            'actual': test_target.values,
            'dates': test_target.index,
            'mse': mse,
            'mae': mae,
            'r2': r2,
            'train_target': train_target,
            'test_target': test_target
        }
        
        print(f"{stock_name} - SARIMAX Performance:")
        print(f"MSE: {mse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"R²: {r2:.4f}")
    else:
        print(f"{stock_name} - SARIMAX model could not be fitted")

print(f"\nSARIMAX models completed for {len(sarimax_results)} stocks")



Processing HDFC Bank for SARIMAX
HDFC Bank - SARIMAX Data Preparation:
Training period: 2022-09-26 00:00:00 to 2025-02-14 00:00:00
Testing period: 2025-02-17 00:00:00 to 2025-09-25 00:00:00
Training samples: 596
Testing samples: 150
HDFC Bank - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
HDFC Bank - Simple SARIMAX Model fitted successfully
HDFC Bank - SARIMAX Performance:
MSE: 123203.9708
MAE: 277.7424
R²: -0.0278

Processing TCS for SARIMAX
TCS - SARIMAX Data Preparation:
Training period: 2022-09-26 00:00:00 to 2025-02-14 00:00:00
Testing period: 2025-02-17 00:00:00 to 2025-09-25 00:00:00
Training samples: 596
Testing samples: 150
TCS - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been prov

TCS - Simple SARIMAX Model fitted successfully
TCS - SARIMAX Performance:
MSE: 388409.3205
MAE: 585.2533
R²: -7.4641

Processing Maruti Suzuki for SARIMAX
Maruti Suzuki - SARIMAX Data Preparation:
Training period: 2024-09-30 00:00:00 to 2025-07-15 00:00:00
Testing period: 2025-07-16 00:00:00 to 2025-09-25 00:00:00
Training samples: 197
Testing samples: 50
Maruti Suzuki - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
Maruti Suzuki - Simple SARIMAX Model fitted successfully
Maruti Suzuki - SARIMAX Performance:
MSE: 3921164.8674
MAE: 1490.3589
R²: -1.1281

Processing Asian Paints for SARIMAX
Asian Paints - SARIMAX Data Preparation:
Training period: 2022-09-26 00:00:00 to 2025-02-14 00:00:00
Testing period: 2025-02-17 00:00:00 to 2025-09-25 00:00:00
Training samples: 596
Testing samples: 150
Asian Paints - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
Asian Paints - Simple 

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it ha

Dabur - Simple SARIMAX Model fitted successfully
Dabur - SARIMAX Performance:
MSE: 918.9512
MAE: 25.2148
R²: -0.7020

Processing Dr. Reddy's for SARIMAX
Dr. Reddy's - SARIMAX Data Preparation:
Training period: 2022-09-26 00:00:00 to 2025-02-14 00:00:00
Testing period: 2025-02-17 00:00:00 to 2025-09-25 00:00:00
Training samples: 596
Testing samples: 150
Dr. Reddy's - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
Dr. Reddy's - Simple SARIMAX Model fitted successfully
Dr. Reddy's - SARIMAX Performance:
MSE: 5716.6904
MAE: 64.6331
R²: -0.2505

Processing Apollo Hospitals for SARIMAX
Apollo Hospitals - SARIMAX Data Preparation:
Training period: 2022-09-26 00:00:00 to 2025-02-17 00:00:00
Testing period: 2025-02-18 00:00:00 to 2025-09-26 00:00:00
Training samples: 597
Testing samples: 150
Apollo Hospitals - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it ha

Apollo Hospitals - Simple SARIMAX Model fitted successfully
Apollo Hospitals - SARIMAX Performance:
MSE: 880981.2936
MAE: 819.6946
R²: -2.5568

Processing Airtel for SARIMAX
Airtel - SARIMAX Data Preparation:
Training period: 2022-09-26 00:00:00 to 2025-02-14 00:00:00
Testing period: 2025-02-17 00:00:00 to 2025-09-25 00:00:00
Training samples: 596
Testing samples: 150
Airtel - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
Airtel - Simple SARIMAX Model fitted successfully
Airtel - SARIMAX Performance:
MSE: 28938.5215
MAE: 152.9930
R²: -1.3477

Processing Motilal Oswal for SARIMAX
Motilal Oswal - SARIMAX Data Preparation:
Training period: 2022-09-26 00:00:00 to 2025-02-17 00:00:00
Testing period: 2025-02-18 00:00:00 to 2025-09-26 00:00:00
Training samples: 597
Testing samples: 150
Motilal Oswal - SARIMAX Model failed: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it ha

Motilal Oswal - Simple SARIMAX Model fitted successfully
Motilal Oswal - SARIMAX Performance:
MSE: 878881.7772
MAE: 818.6595
R²: -2.5483

SARIMAX models completed for 9 stocks


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.



In [9]:
# Part 2: Support Vector Regression (SVR) Implementation

def prepare_svr_data(df, stock_name, target_col='CLOSE'):
    df_processed = df.copy()
    df_processed['Open-Close'] = df_processed['OPEN'] - df_processed['CLOSE']
    df_processed['High-Low'] = df_processed['HIGH'] - df_processed['LOW']
    df_processed['Price_Change'] = df_processed['CLOSE'].pct_change()
    df_processed['SMA_5'] = df_processed['CLOSE'].rolling(window=5).mean()
    df_processed['SMA_10'] = df_processed['CLOSE'].rolling(window=10).mean()
    df_processed['Volatility'] = df_processed['Price_Change'].rolling(window=10).std()
    
    feature_cols = ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
    X = df_processed[feature_cols].fillna(method='ffill').fillna(0)
    y = df_processed[target_col]
    
    valid_indices = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[valid_indices]
    y = y[valid_indices]
    
    split_point = int(0.8 * len(X))
    
    X_train = X[:split_point]
    X_test = X[split_point:]
    y_train = y[:split_point]
    y_test = y[split_point:]
    
    print(f"{stock_name} - SVR Data Preparation:")
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    print(f"Features: {list(X.columns)}")
    
    return X_train, X_test, y_train, y_test, X.index[split_point:]

def train_svr_model(X_train, X_test, y_train, y_test, stock_name):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    svr = SVR(kernel='rbf', C=1.0, gamma='scale', epsilon=0.1)
    
    svr.fit(X_train_scaled, y_train)
    y_pred_train = svr.predict(X_train_scaled)
    y_pred_test = svr.predict(X_test_scaled)
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    print(f"{stock_name} - SVR Performance:")
    print(f"Training MSE: {train_mse:.4f}, MAE: {train_mae:.4f}, R²: {train_r2:.4f}")
    print(f"Testing MSE: {test_mse:.4f}, MAE: {test_mae:.4f}, R²: {test_r2:.4f}")
    
    return {
        'model': svr,
        'scaler': scaler,
        'predictions': y_pred_test,
        'actual': y_test.values,
        'mse': test_mse,
        'mae': test_mae,
        'r2': test_r2,
        'train_predictions': y_pred_train,
        'train_actual': y_train.values
    }

svr_results = {}

for stock_name, df in stocks_data.items():
    print(f"\n{'='*50}")
    print(f"Processing {stock_name} for SVR")
    print(f"{'='*50}")
    X_train, X_test, y_train, y_test, test_dates = prepare_svr_data(df, stock_name)
    result = train_svr_model(X_train, X_test, y_train, y_test, stock_name)
    result['dates'] = test_dates
    
    svr_results[stock_name] = result

print(f"\nSVR models completed for {len(svr_results)} stocks")



Processing HDFC Bank for SVR
HDFC Bank - SVR Data Preparation:
Training samples: 596
Testing samples: 150
Features: ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
HDFC Bank - SVR Performance:
Training MSE: 21870.0584, MAE: 71.9167, R²: 0.1731
Testing MSE: 117609.9247, MAE: 298.6715, R²: 0.0189

Processing TCS for SVR
TCS - SVR Data Preparation:
Training samples: 596
Testing samples: 150
Features: ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
TCS - SVR Performance:
Training MSE: 127728.2410, MAE: 309.7556, R²: 0.2522
Testing MSE: 105425.3350, MAE: 273.2342, R²: -1.2974

Processing Maruti Suzuki for SVR
Maruti Suzuki - SVR Data Preparation:
Training samples: 197
Testing samples: 50
Features: ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
Maruti Suzuki - SVR Performance:
Training MSE: 398648.9350, MAE: 546.6676, R²: 0.0133
Testing MSE: 5858694.1173, MAE: 2006.0516, R²: -2.1796

Processing Asian Paints

In [ ]:
# Fixed Comprehensive Visualization of All Models

def create_fixed_comprehensive_visualization():
    # Recreate comparison data within the function to ensure proper access
    all_results = {
        'SARIMAX': sarimax_results,
        'SVR': svr_results,
        'XGBoost': xgboost_results,
        'LSTM': lstm_results
    }
    
    comparison_data = []
    
    for model_name, results in all_results.items():
        for stock_name, result in results.items():
            comparison_data.append({
                'Model': model_name,
                'Stock': stock_name,
                'MSE': result['mse'],
                'MAE': result['mae'],
                'R²': result['r2']
            })
    
    comparison_df_local = pd.DataFrame(comparison_data)
    print(f"Created comparison dataframe with {len(comparison_df_local)} rows")
    print(f"Available models: {comparison_df_local['Model'].unique()}")
    print(f"Available stocks: {comparison_df_local['Stock'].unique()}")
    
    # Create subplots
    fig = make_subplots(
        rows=4, cols=2,
        subplot_titles=[
            'Model Performance Comparison (MSE)',
            'Model Performance Comparison (R²)',
            'Actual vs Predicted - SARIMAX',
            'Actual vs Predicted - SVR',
            'Actual vs Predicted - XGBoost',
            'Actual vs Predicted - LSTM',
            'Model Accuracy by Stock (R² Heatmap)',
            'Training Loss Curves (LSTM)'
        ],
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Model Performance Comparison (MSE) - Fixed
    mse_data = comparison_df_local.groupby('Model')['MSE'].mean().sort_values()
    print(f"MSE data: {mse_data}")
    fig.add_trace(
        go.Bar(x=mse_data.index, y=mse_data.values, name='Average MSE',
               marker_color='lightcoral', showlegend=False),
        row=1, col=1
    )
    
    # 2. Model Performance Comparison (R²) - Fixed
    r2_data = comparison_df_local.groupby('Model')['R²'].mean().sort_values(ascending=False)
    print(f"R² data: {r2_data}")
    fig.add_trace(
        go.Bar(x=r2_data.index, y=r2_data.values, name='Average R²',
               marker_color='lightgreen', showlegend=False),
        row=1, col=2
    )
    
    # 3-6. Actual vs Predicted plots for each model
    models_data = {
        'SARIMAX': sarimax_results,
        'SVR': svr_results,
        'XGBoost': xgboost_results,
        'LSTM': lstm_results
    }
    
    positions = [(2, 1), (2, 2), (3, 1), (3, 2)]
    
    for i, (model_name, results) in enumerate(models_data.items()):
        if results and i < len(positions):
            row, col = positions[i]
            stock_name = list(results.keys())[0]  # Take first stock
            result = results[stock_name]
            
            actual = result['actual']
            predicted = result['predictions']
            
            fig.add_trace(
                go.Scatter(x=actual, y=predicted, mode='markers',
                          name=f'{model_name} - {stock_name}',
                          marker=dict(size=6, opacity=0.6),
                          showlegend=False),
                row=row, col=col
            )
            # Add perfect prediction line
            min_val = min(min(actual), min(predicted))
            max_val = max(max(actual), max(predicted))
            fig.add_trace(
                go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                          mode='lines', name='Perfect Prediction',
                          line=dict(dash='dash', color='red'),
                          showlegend=False),
                row=row, col=col
            )
    
    # 7. Model Accuracy by Stock (R² Heatmap) - Fixed
    pivot_data = comparison_df_local.pivot(index='Stock', columns='Model', values='R²')
    print(f"Pivot data shape: {pivot_data.shape}")
    print(f"Pivot data:\n{pivot_data}")
    
    fig.add_trace(
        go.Heatmap(z=pivot_data.values,
                  x=pivot_data.columns,
                  y=pivot_data.index,
                  colorscale='RdYlGn',
                  showscale=True,
                  name='R² Score'),
        row=4, col=1
    )
    
    # 8. Training Loss Curves (LSTM)
    if lstm_results:
        stock_name = list(lstm_results.keys())[0]
        history = lstm_results[stock_name]['history']
        
        fig.add_trace(
            go.Scatter(y=history.history['loss'], mode='lines',
                      name='Training Loss', line=dict(color='blue')),
            row=4, col=2
        )
        
        fig.add_trace(
            go.Scatter(y=history.history['val_loss'], mode='lines',
                      name='Validation Loss', line=dict(color='red')),
            row=4, col=2
        )
    
    # Update layout
    fig.update_layout(
        title_text="Fixed Comprehensive Stock Price Prediction Analysis",
        title_x=0.5,
        height=1600,
        showlegend=True
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Models", row=1, col=1)
    fig.update_yaxes(title_text="MSE", row=1, col=1)
    
    fig.update_xaxes(title_text="Models", row=1, col=2)
    fig.update_yaxes(title_text="R²", row=1, col=2)
    
    for row in [2, 3]:
        for col in [1, 2]:
            fig.update_xaxes(title_text="Actual Price", row=row, col=col)
            fig.update_yaxes(title_text="Predicted Price", row=row, col=col)
    
    fig.update_xaxes(title_text="Models", row=4, col=1)
    fig.update_yaxes(title_text="Stocks", row=4, col=1)
    
    fig.update_xaxes(title_text="Epochs", row=4, col=2)
    fig.update_yaxes(title_text="Loss", row=4, col=2)
    
    fig.show()
    
    return comparison_df_local

# Call the fixed visualization function
if 'sarimax_results' in locals() and 'svr_results' in locals() and 'xgboost_results' in locals() and 'lstm_results' in locals():
    comparison_df_fixed = create_fixed_comprehensive_visualization()
else:
    print("Some models are not yet trained. Please run all model training cells first.")


Some models are not yet trained. Please run all model training cells first.


In [11]:
# Part 2: XGBoost Implementation

def train_xgboost_model(X_train, X_test, y_train, y_test, stock_name):
    xgb_model = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    xgb_model.fit(X_train, y_train)
    y_pred_train = xgb_model.predict(X_train)
    y_pred_test = xgb_model.predict(X_test)
    
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    print(f"{stock_name} - XGBoost Performance:")
    print(f"Training MSE: {train_mse:.4f}, MAE: {train_mae:.4f}, R²: {train_r2:.4f}")
    print(f"Testing MSE: {test_mse:.4f}, MAE: {test_mae:.4f}, R²: {test_r2:.4f}")
    
    return {
        'model': xgb_model,
        'predictions': y_pred_test,
        'actual': y_test.values,
        'mse': test_mse,
        'mae': test_mae,
        'r2': test_r2,
        'train_predictions': y_pred_train,
        'train_actual': y_train.values
    }
xgboost_results = {}

for stock_name, df in stocks_data.items():
    print(f"\n{'='*50}")
    print(f"Processing {stock_name} for XGBoost")
    print(f"{'='*50}")
    
    X_train, X_test, y_train, y_test, test_dates = prepare_svr_data(df, stock_name)
    
    result = train_xgboost_model(X_train, X_test, y_train, y_test, stock_name)
    result['dates'] = test_dates
    
    xgboost_results[stock_name] = result

print(f"\nXGBoost models completed for {len(xgboost_results)} stocks")



Processing HDFC Bank for XGBoost
HDFC Bank - SVR Data Preparation:
Training samples: 596
Testing samples: 150
Features: ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
HDFC Bank - XGBoost Performance:
Training MSE: 33.1675, MAE: 4.4380, R²: 0.9987
Testing MSE: 42612.3614, MAE: 139.0798, R²: 0.6445

Processing TCS for XGBoost
TCS - SVR Data Preparation:
Training samples: 596
Testing samples: 150
Features: ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
TCS - XGBoost Performance:
Training MSE: 67.6325, MAE: 6.4897, R²: 0.9996
Testing MSE: 3696.9848, MAE: 42.9922, R²: 0.9194

Processing Maruti Suzuki for XGBoost
Maruti Suzuki - SVR Data Preparation:
Training samples: 197
Testing samples: 50
Features: ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
Maruti Suzuki - XGBoost Performance:
Training MSE: 87.4184, MAE: 7.2261, R²: 0.9998
Testing MSE: 3536886.3558, MAE: 1386.3183, R²: -0.9195

Processing Asian Pai

In [12]:
# Part 2: Neural Network Implementation (LSTM/GRU)

def prepare_lstm_data(df, stock_name, target_col='CLOSE', sequence_length=60):
    df_processed = df.copy()
    df_processed['Open-Close'] = df_processed['OPEN'] - df_processed['CLOSE']
    df_processed['High-Low'] = df_processed['HIGH'] - df_processed['LOW']
    df_processed['Price_Change'] = df_processed['CLOSE'].pct_change()
    df_processed['SMA_5'] = df_processed['CLOSE'].rolling(window=5).mean()
    df_processed['SMA_10'] = df_processed['CLOSE'].rolling(window=10).mean()
    df_processed['Volatility'] = df_processed['Price_Change'].rolling(window=10).std()
    
    feature_cols = ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
    features = df_processed[feature_cols].fillna(method='ffill').fillna(0).values
    target = df_processed[target_col].fillna(method='ffill').values
    
    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler()
    
    features_scaled = scaler_features.fit_transform(features)
    target_scaled = scaler_target.fit_transform(target.reshape(-1, 1)).flatten()
    
    X, y = [], []
    for i in range(sequence_length, len(features_scaled)):
        X.append(features_scaled[i-sequence_length:i])
        y.append(target_scaled[i])
    
    X, y = np.array(X), np.array(y)
    
    split_point = int(0.8 * len(X))
    
    X_train = X[:split_point]
    X_test = X[split_point:]
    y_train = y[:split_point]
    y_test = y[split_point:]
    
    print(f"{stock_name} - LSTM Data Preparation:")
    print(f"Sequence length: {sequence_length}")
    print(f"Training samples: {len(X_train)}")
    print(f"Testing samples: {len(X_test)}")
    print(f"Feature shape: {X_train.shape}")
    
    return X_train, X_test, y_train, y_test, scaler_features, scaler_target

def create_lstm_model(input_shape, stock_name):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    
    print(f"{stock_name} - LSTM Model created:")
    print(f"Input shape: {input_shape}")
    model.summary()
    
    return model

def train_lstm_model(X_train, X_test, y_train, y_test, scaler_target, stock_name):
    model = create_lstm_model((X_train.shape[1], X_train.shape[2]), stock_name)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=50,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )
    y_pred_train = model.predict(X_train, verbose=0)
    y_pred_test = model.predict(X_test, verbose=0)
    y_pred_train = scaler_target.inverse_transform(y_pred_train).flatten()
    y_pred_test = scaler_target.inverse_transform(y_pred_test).flatten()
    y_train_actual = scaler_target.inverse_transform(y_train.reshape(-1, 1)).flatten()
    y_test_actual = scaler_target.inverse_transform(y_test.reshape(-1, 1)).flatten()
    train_mse = mean_squared_error(y_train_actual, y_pred_train)
    test_mse = mean_squared_error(y_test_actual, y_pred_test)
    train_mae = mean_absolute_error(y_train_actual, y_pred_train)
    test_mae = mean_absolute_error(y_test_actual, y_pred_test)
    train_r2 = r2_score(y_train_actual, y_pred_train)
    test_r2 = r2_score(y_test_actual, y_pred_test)
    
    print(f"{stock_name} - LSTM Performance:")
    print(f"Training MSE: {train_mse:.4f}, MAE: {train_mae:.4f}, R²: {train_r2:.4f}")
    print(f"Testing MSE: {test_mse:.4f}, MAE: {test_mae:.4f}, R²: {test_r2:.4f}")
    
    return {
        'model': model,
        'history': history,
        'predictions': y_pred_test,
        'actual': y_test_actual,
        'mse': test_mse,
        'mae': test_mae,
        'r2': test_r2,
        'train_predictions': y_pred_train,
        'train_actual': y_train_actual
    }
lstm_results = {}

for stock_name, df in stocks_data.items():
    print(f"\n{'='*50}")
    print(f"Processing {stock_name} for LSTM")
    print(f"{'='*50}")
    
    try:
        X_train, X_test, y_train, y_test, scaler_features, scaler_target = prepare_lstm_data(df, stock_name)
        result = train_lstm_model(X_train, X_test, y_train, y_test, scaler_target, stock_name)
        
        lstm_results[stock_name] = result
        
    except Exception as e:
        print(f"{stock_name} - LSTM training failed: {str(e)}")

print(f"\nLSTM models completed for {len(lstm_results)} stocks")



Processing HDFC Bank for LSTM
HDFC Bank - LSTM Data Preparation:
Sequence length: 60
Training samples: 548
Testing samples: 138
Feature shape: (548, 60, 6)
HDFC Bank - LSTM Model created:
Input shape: (60, 6)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 50)         │        11,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,901 (128.52 KB)

 Trainable params: 32,901 (128.52 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                  │ (None, 60, 50)         │        11,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,901 (128.52 KB)

 Trainable params: 32,901 (128.52 KB)

 Non-trainable params: 0 (0.00 B)

Motilal Oswal - LSTM Performance:
Training MSE: 226092.6446, MAE: 171.4908, R²: 0.9607
Testing MSE: 27368.6329, MAE: 128.1366, R²: 0.8643

LSTM models completed for 9 stocks


In [ ]:
# Model Performance Comparison and Analysis

def compare_model_performance():
    print("="*80)
    print("MODEL PERFORMANCE COMPARISON")
    print("="*80)
    
    # Collect all results
    all_results = {
        'SARIMAX': sarimax_results,
        'SVR': svr_results,
        'XGBoost': xgboost_results,
        'LSTM': lstm_results
    }
    
    comparison_data = []
    
    for model_name, results in all_results.items():
        for stock_name, result in results.items():
            comparison_data.append({
                'Model': model_name,
                'Stock': stock_name,
                'MSE': result['mse'],
                'MAE': result['mae'],
                'R²': result['r2']
            })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    print("\nSUMMARY STATISTICS BY MODEL:")
    print("-" * 50)
    summary_stats = comparison_df.groupby('Model').agg({
        'MSE': ['mean', 'std'],
        'MAE': ['mean', 'std'],
        'R²': ['mean', 'std']
    }).round(4)
    
    print(summary_stats)
    print("\nBEST MODEL FOR EACH STOCK (by R²):")
    print("-" * 50)
    best_models = comparison_df.loc[comparison_df.groupby('Stock')['R²'].idxmax()]
    for _, row in best_models.iterrows():
        print(f"{row['Stock']}: {row['Model']} (R² = {row['R²']:.4f})")
    print("\nOVERALL BEST MODEL (by average R²):")
    print("-" * 50)
    avg_r2 = comparison_df.groupby('Model')['R²'].mean().sort_values(ascending=False)
    for model, r2 in avg_r2.items():
        print(f"{model}: {r2:.4f}")
    
    return comparison_df
if 'sarimax_results' in locals() and 'svr_results' in locals() and 'xgboost_results' in locals() and 'lstm_results' in locals():
    comparison_df = compare_model_performance()
else:
    print("Some models are not yet trained. Please run all model training cells first.")


MODEL PERFORMANCE COMPARISON

SUMMARY STATISTICS BY MODEL:
--------------------------------------------------
                  MSE                      MAE                   R²           
                 mean           std       mean        std      mean        std
Model                                                                         
LSTM     5.071824e+05  1.468552e+06   270.8329   572.0097    0.3370     0.8713
SARIMAX  6.955856e+05  1.261818e+06   486.9596   487.2009   -1.9745     2.2458
SVR      6.989413e+06  1.246057e+07  1721.0563  2056.2834 -427.4755  1197.3509
XGBoost  4.680747e+05  1.157739e+06   291.8245   425.6156  -10.0084    29.5085

BEST MODEL FOR EACH STOCK (by R²):
--------------------------------------------------
Airtel: LSTM (R² = 0.8139)
Apollo Hospitals: LSTM (R² = 0.8259)
Asian Paints: LSTM (R² = 0.4866)
Dabur: LSTM (R² = 0.3611)
Dr. Reddy's: LSTM (R² = 0.4378)
HDFC Bank: XGBoost (R² = 0.6445)
Maruti Suzuki: XGBoost (R² = -0.9195)
Motilal Oswal: LSTM (R² 

## Advanced Model: Hybrid ARIMA-TDNN Approach

In this section, we will implement a hybrid approach combining ARIMA with Time Delay Neural Networks (TDNN) as mentioned in the project requirements. This approach leverages the strengths of both time series analysis (ARIMA) and deep learning with temporal delays (TDNN).


In [ ]:
def create_hybrid_arima_tdnn_model(df, stock_name, target_col='CLOSE', sequence_length=60):
    """
    Create a hybrid ARIMA-TDNN model that combines ARIMA for linear patterns
    and TDNN for capturing non-linear residuals with temporal dependencies
    """
    print(f"\n{'='*60}")
    print(f"Creating Hybrid ARIMA-TDNN Model for {stock_name}")
    print(f"{'='*60}")
    
    df_processed = df.copy()
    df_processed.index = pd.to_datetime(df_processed['Date'])
    
    df_processed['Open-Close'] = df_processed['OPEN'] - df_processed['CLOSE']
    df_processed['High-Low'] = df_processed['HIGH'] - df_processed['LOW']
    df_processed['Price_Change'] = df_processed['CLOSE'].pct_change()
    df_processed['SMA_5'] = df_processed['CLOSE'].rolling(window=5).mean()
    df_processed['SMA_10'] = df_processed['CLOSE'].rolling(window=10).mean()
    df_processed['Volatility'] = df_processed['Price_Change'].rolling(window=10).std()
    
    target_data = df_processed[target_col].dropna()
    
    split_point = int(0.8 * len(target_data))
    train_target = target_data[:split_point]
    test_target = target_data[split_point:]
    
    # Step 1: Fit ARIMA model
    try:
        arima_model = ARIMA(train_target, order=(1, 1, 1))
        arima_fitted = arima_model.fit()
        
        arima_pred_train = arima_fitted.fittedvalues
        arima_pred_test = arima_fitted.forecast(steps=len(test_target))
        
        print(f"ARIMA model fitted successfully")
        print(f"ARIMA AIC: {arima_fitted.aic:.2f}")
        
    except Exception as e:
        print(f"ARIMA model failed: {str(e)}")
        arima_pred_train = train_target.rolling(window=5).mean().fillna(train_target.mean())
        arima_pred_test = np.full(len(test_target), train_target.mean())
    
    # Calculate ARIMA residuals
    arima_residuals_train = train_target - arima_pred_train
    arima_residuals_test = test_target - arima_pred_test
    
    # Step 2: Prepare TDNN data for residual prediction
    feature_cols = ['Open-Close', 'High-Low', 'Price_Change', 'SMA_5', 'SMA_10', 'Volatility']
    features = df_processed[feature_cols].fillna(method='ffill').fillna(0)

    common_index = target_data.index.intersection(features.index)
    features_aligned = features.loc[common_index].values
    residuals_aligned_train = arima_residuals_train.loc[common_index[:split_point]]
    residuals_aligned_test = arima_residuals_test.loc[common_index[split_point:]]
    
    # Scale features for TDNN
    scaler_features = MinMaxScaler()
    scaler_residuals = MinMaxScaler()
    
    features_scaled = scaler_features.fit_transform(features_aligned)
    residuals_scaled_train = scaler_residuals.fit_transform(residuals_aligned_train.values.reshape(-1, 1)).flatten()
    residuals_scaled_test = scaler_residuals.transform(residuals_aligned_test.values.reshape(-1, 1)).flatten()
    
    # Create TDNN sequences: flatten time-delayed windows
    X_train, y_train_tdnn = [], []
    X_test, y_test_tdnn = [], []
    
    for i in range(sequence_length, split_point):
        X_train.append(features_scaled[i-sequence_length:i].flatten())
        y_train_tdnn.append(residuals_scaled_train[i])
    
    for i in range(sequence_length, len(residuals_scaled_test)):
        test_idx = split_point + i - sequence_length
        if test_idx < len(features_scaled):
            X_test.append(features_scaled[test_idx-sequence_length:test_idx].flatten())
            y_test_tdnn.append(residuals_scaled_test[i])
    
    X_train, y_train_tdnn = np.array(X_train), np.array(y_train_tdnn)
    X_test, y_test_tdnn = np.array(X_test), np.array(y_test_tdnn)
    
    # Adjust test target to match TDNN output length
    if len(y_test_tdnn) < len(test_target):
        test_target_adjusted = test_target.iloc[:len(y_test_tdnn)]
        arima_pred_test_adjusted = arima_pred_test[:len(y_test_tdnn)]
    else:
        test_target_adjusted = test_target
        arima_pred_test_adjusted = arima_pred_test
    
    # Step 3: Create and train TDNN model
    def create_tdnn_model(input_dim):
        model = Sequential([
            Dense(128, activation='relu', input_shape=(input_dim,)),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(32, activation='relu'),
            Dropout(0.2),
            Dense(16, activation='relu'),
            Dense(1)
        ])
        
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
        return model
    
    if len(X_train) > 0:
        tdnn_model = create_tdnn_model(X_train.shape[1])
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
        
        history = tdnn_model.fit(
            X_train, y_train_tdnn,
            epochs=50,
            batch_size=32,
            validation_split=0.2,
            callbacks=[early_stopping, reduce_lr],
            verbose=0
        )
        
        tdnn_residuals_train_scaled = tdnn_model.predict(X_train, verbose=0).flatten()
        tdnn_residuals_test_scaled = tdnn_model.predict(X_test, verbose=0).flatten()
        
        # Inverse transform TDNN residuals
        tdnn_residuals_train = scaler_residuals.inverse_transform(tdnn_residuals_train_scaled.reshape(-1, 1)).flatten()
        tdnn_residuals_test = scaler_residuals.inverse_transform(tdnn_residuals_test_scaled.reshape(-1, 1)).flatten()
        
        # Adjust train predictions to match length
        if len(tdnn_residuals_train) < len(train_target):
            train_target_adjusted = train_target.iloc[-len(tdnn_residuals_train):]
            arima_pred_train_adjusted = arima_pred_train.iloc[-len(tdnn_residuals_train):]
            hybrid_pred_train = arima_pred_train_adjusted + tdnn_residuals_train
        else:
            train_target_adjusted = train_target
            hybrid_pred_train = arima_pred_train + tdnn_residuals_train[:len(train_target)]
        
        hybrid_pred_test = arima_pred_test_adjusted + tdnn_residuals_test
        
        # Calculate metrics
        train_mse = mean_squared_error(train_target_adjusted, hybrid_pred_train)
        test_mse = mean_squared_error(test_target_adjusted, hybrid_pred_test)
        train_mae = mean_absolute_error(train_target_adjusted, hybrid_pred_train)
        test_mae = mean_absolute_error(test_target_adjusted, hybrid_pred_test)
        train_r2 = r2_score(train_target_adjusted, hybrid_pred_train)
        test_r2 = r2_score(test_target_adjusted, hybrid_pred_test)
    else:
        print("Warning: Insufficient data for TDNN. Using ARIMA predictions only.")
        hybrid_pred_train = arima_pred_train
        hybrid_pred_test = arima_pred_test
        train_target_adjusted = train_target
        test_target_adjusted = test_target
        arima_pred_test_adjusted = arima_pred_test
        history = None
        tdnn_model = None
        tdnn_residuals_test = None
        
        # Calculate metrics using ARIMA only
        train_mse = mean_squared_error(train_target_adjusted, hybrid_pred_train)
        test_mse = mean_squared_error(test_target_adjusted, hybrid_pred_test)
        train_mae = mean_absolute_error(train_target_adjusted, hybrid_pred_train)
        test_mae = mean_absolute_error(test_target_adjusted, hybrid_pred_test)
        train_r2 = r2_score(train_target_adjusted, hybrid_pred_train)
        test_r2 = r2_score(test_target_adjusted, hybrid_pred_test)
    
    print(f"\nHybrid ARIMA-TDNN Performance for {stock_name}:")
    print(f"Training MSE: {train_mse:.4f}, MAE: {train_mae:.4f}, R²: {train_r2:.4f}")
    print(f"Testing MSE: {test_mse:.4f}, MAE: {test_mae:.4f}, R²: {test_r2:.4f}")
    
    return {
        'arima_model': arima_fitted if 'arima_fitted' in locals() else None,
        'tdnn_model': tdnn_model,
        'history': history,
        'predictions': hybrid_pred_test,
        'actual': test_target_adjusted.values,
        'mse': test_mse,
        'mae': test_mae,
        'r2': test_r2,
        'train_predictions': hybrid_pred_train,
        'train_actual': train_target_adjusted.values,
        'arima_predictions': arima_pred_test_adjusted,
        'tdnn_residuals': tdnn_residuals_test if 'tdnn_residuals_test' in locals() and tdnn_residuals_test is not None else None
    }

# Initialize dictionary to store results for all stocks
hybrid_results_all = {}

# Process all stocks
for stock_name, df in stocks_data.items():
    try:
        print(f"\n{'='*80}")
        print(f"Processing {stock_name} for Hybrid ARIMA-TDNN")
        print(f"{'='*80}")
        
        hybrid_result = create_hybrid_arima_tdnn_model(df, stock_name)
        hybrid_results_all[stock_name] = hybrid_result
        
        print(f"Hybrid ARIMA-TDNN model completed for {stock_name}")
        
    except Exception as e:
        print(f"Error processing {stock_name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*80}")
print(f"HYBRID ARIMA-TDNN SUMMARY")
print(f"{'='*80}")
print(f"Successfully processed {len(hybrid_results_all)} stocks")

for stock_name, result in hybrid_results_all.items():
    print(f"{stock_name}: MSE={result['mse']:.4f}, MAE={result['mae']:.4f}, R²={result['r2']:.4f}")



Processing HDFC Bank for Hybrid ARIMA-ANN

Creating Hybrid ARIMA-ANN Model for HDFC Bank
ARIMA model fitted successfully
ARIMA AIC: 7353.35


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.




Hybrid ARIMA-ANN Performance for HDFC Bank:
Training MSE: 5927.7098, MAE: 27.9097, R²: 0.7759
Testing MSE: 121179.9693, MAE: 257.6763, R²: -0.0109
Hybrid ARIMA-ANN model completed for HDFC Bank

Processing TCS for Hybrid ARIMA-ANN

Creating Hybrid ARIMA-ANN Model for TCS
ARIMA model fitted successfully
ARIMA AIC: 6272.98


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.




Hybrid ARIMA-ANN Performance for TCS:
Training MSE: 11943.2083, MAE: 17.7770, R²: 0.9301
Testing MSE: 391177.6496, MAE: 586.1670, R²: -7.5244
Hybrid ARIMA-ANN model completed for TCS

Processing Maruti Suzuki for Hybrid ARIMA-ANN

Creating Hybrid ARIMA-ANN Model for Maruti Suzuki
ARIMA model fitted successfully
ARIMA AIC: 2540.62


...


Hybrid ARIMA-ANN Performance for Apollo Hospitals:
Training MSE: 24405.3966, MAE: 27.2029, R²: 0.9752
Testing MSE: 854256.5483, MAE: 805.0058, R²: -2.4489
Hybrid ARIMA-ANN model completed for Apollo Hospitals

Processing Airtel for Hybrid ARIMA-ANN

Creating Hybrid ARIMA-ANN Model for Airtel
ARIMA model fitted successfully
ARIMA AIC: 4970.25


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.




Hybrid ARIMA-ANN Performance for Airtel:
Training MSE: 808.8013, MAE: 4.7687, R²: 0.9925
Testing MSE: 26699.3817, MAE: 146.6279, R²: -1.1660
Hybrid ARIMA-ANN model completed for Airtel

Processing Motilal Oswal for Hybrid ARIMA-ANN

Creating Hybrid ARIMA-ANN Model for Motilal Oswal
ARIMA model fitted successfully
ARIMA AIC: 8412.48


c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

c:\Users\home\miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.




Hybrid ARIMA-ANN Performance for Motilal Oswal:
Training MSE: 62186.4448, MAE: 32.4254, R²: 0.9879
Testing MSE: 940054.3047, MAE: 845.3779, R²: -2.7953
Hybrid ARIMA-ANN model completed for Motilal Oswal

HYBRID ARIMA-ANN SUMMARY
Successfully processed 9 stocks
HDFC Bank: MSE=121179.9693, MAE=257.6763, R²=-0.0109
TCS: MSE=391177.6496, MAE=586.1670, R²=-7.5244
Maruti Suzuki: MSE=4023038.3953, MAE=1500.0107, R²=-1.1834
Asian Paints: MSE=24208.0410, MAE=124.7429, R²=-1.0730
Dabur: MSE=1128.8453, MAE=27.9038, R²=-1.0908
Dr. Reddy's: MSE=54709.3412, MAE=221.4848, R²=-10.9673
Apollo Hospitals: MSE=854256.5483, MAE=805.0058, R²=-2.4489
Airtel: MSE=26699.3817, MAE=146.6279, R²=-1.1660
Motilal Oswal: MSE=940054.3047, MAE=845.3779, R²=-2.7953


In [20]:
print("\nAll processes completed.")


All processes completed.


This project has given a comprehensive implementation of various stock price prediction models including SARIMAX, SVR, XGBoost, LSTM, TDNN, and a hybrid ARIMA-TDNN model. Each model has been trained, evaluated, and visualized to provide insights into their performance. The visualizations help in comparing the effectiveness of each model across different stocks. Further enhancements can include hyperparameter tuning, ensemble methods, and real-time data integration for more robust predictions.

**Key Models Implemented:**
1. SARIMAX - Seasonal ARIMA with exogenous variables
2. SVR - Support Vector Regression
3. XGBoost - Gradient Boosting for regression
4. LSTM - Long Short-Term Memory networks
5. TDNN - Time Delay Neural Networks
6. Hybrid ARIMA-TDNN - Combines ARIMA for linear patterns with TDNN for non-linear residual modeling

The implication of all results and why they occur for different stocks will be explained in detail at ppt with proper academic research.
